In [ ]:
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.conditions import TextMentionTermination
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.ui import Console
from autogen_core.tools import FunctionTool
from autogen_ext.models.openai import OpenAIChatCompletionClient

def google_search(query: str, num_results: int = 2, max_chars: int = 500) -> list:  # type: ignore[type-arg]
    import os
    import time

    import requests
    from bs4 import BeautifulSoup
    from dotenv import load_dotenv

    load_dotenv()

    api_key = os.getenv("GOOGLE_API_KEY")
    search_engine_id = os.getenv("GOOGLE_SEARCH_ENGINE_ID")

    if not api_key or not search_engine_id:
        raise ValueError("API key or Search Engine ID not found in environment variables")

    url = "https://www.googleapis.com/customsearch/v1"
    params = {"key": api_key, "cx": search_engine_id, "q": query, "num": num_results}

    response = requests.get(url, params=params)  # type: ignore[arg-type]

    if response.status_code != 200:
        print(response.json())
        raise Exception(f"Error in API request: {response.status_code}")

    results = response.json().get("items", [])

    def get_page_content(url: str) -> str:
        try:
            response = requests.get(url, timeout=10)
            soup = BeautifulSoup(response.content, "html.parser")
            text = soup.get_text(separator=" ", strip=True)
            words = text.split()
            content = ""
            for word in words:
                if len(content) + len(word) + 1 > max_chars:
                    break
                content += " " + word
            return content.strip()
        except Exception as e:
            print(f"Error fetching {url}: {str(e)}")
            return ""

    enriched_results = []
    for item in results:
        body = get_page_content(item["link"])
        enriched_results.append(
            {"title": item["title"], "link": item["link"], "snippet": item["snippet"], "body": body}
        )
        time.sleep(1)  # Be respectful to the servers

    return enriched_results


def arxiv_search(query: str, max_results: int = 2) -> list:  # type: ignore[type-arg]
    """
    Search Arxiv for papers and return the results including abstracts.
    """
    import arxiv

    client = arxiv.Client()
    search = arxiv.Search(query=query, max_results=max_results, sort_by=arxiv.SortCriterion.Relevance)

    results = []
    for paper in client.results(search):
        results.append(
            {
                "title": paper.title,
                "authors": [author.name for author in paper.authors],
                "published": paper.published.strftime("%Y-%m-%d"),
                "abstract": paper.summary,
                "pdf_url": paper.pdf_url,
            }
        )

    # # Write results to a file
    # with open('arxiv_search_results.json', 'w') as f:
    #     json.dump(results, f, indent=2)

    return results


google_search_tool = FunctionTool(
    google_search, description="Search Google for information, returns results with a snippet and body content"
)
arxiv_search_tool = FunctionTool(
    arxiv_search, description="Search Arxiv for papers related to a given topic, including abstracts"
)


google_search_agent = AssistantAgent(
    name="Google_Search_Agent",
    tools=[google_search_tool],
    model_client=OpenAIChatCompletionClient(model="gpt-4o-mini"),
    description="An agent that can search Google for information, returns results with a snippet and body content",
    system_message="You are a helpful AI assistant. Solve tasks using your tools.",
)

arxiv_search_agent = AssistantAgent(
    name="Arxiv_Search_Agent",
    tools=[arxiv_search_tool],
    model_client=OpenAIChatCompletionClient(model="gpt-4o-mini"),
    description="An agent that can search Arxiv for papers related to a given topic, including abstracts",
    system_message="You are a helpful AI assistant. Solve tasks using your tools. Specifically, you can take into consideration the user's request and craft a search query that is most likely to return relevant academi papers.",
)


report_agent = AssistantAgent(
    name="Report_Agent",
    model_client=OpenAIChatCompletionClient(model="gpt-4o-mini"),
    description="Generate a report based on a given topic",
    system_message="You are a helpful assistant. Your task is to synthesize data extracted into a high quality literature review including CORRECT references. You MUST write a final report that is formatted as a literature review with CORRECT references.  Your response should end with the word 'TERMINATE'",
)


termination = TextMentionTermination("TERMINATE")
team = RoundRobinGroupChat(
    participants=[google_search_agent, arxiv_search_agent, report_agent], termination_condition=termination
)

await Console(
    team.run_stream(
        task="Write a literature review on no code tools for building multi agent ai systems",
    )
)

In [22]:
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.conditions import TextMentionTermination
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.ui import Console
from autogen_core.tools import FunctionTool
import google.generativeai as genai
from typing import List, Optional, Union, Dict
import os
import json
import asyncio

class GeminiChatCompletionClient:
    def __init__(self, model: str = "gemini-pro"):
        """Initialize Gemini client with API key and model."""
        genai.configure(api_key=os.getenv("GEMINI_API_KEY"))
        self.model = genai.GenerativeModel(model)
        self.chat = self.model.start_chat()
        
        # Complete model_info with all required fields
        self.model_info = {
            "name": model,
            "function_calling": True,
            "max_tokens": 30720,
            "context_window": 30720,
            "max_output_tokens": 2048,
            "vision": False,  # Added vision field
            "json_mode": False,  # Added json_mode field
            "seed_required": False,  # Added seed_required field
            "response_format": {"type": "text"},  # Added response_format field
            "supports_functions": True,  # Added supports_functions field
            "supports_vision": False,  # Added supports_vision field
            "max_function_calls": 50  # Added max_function_calls field
        }
        
        # Required configuration attributes
        self.config = {
            "temperature": 0.7,
            "top_p": 0.95,
            "timeout": 60,
            "seed": None,  # Added seed field
            "response_format": {"type": "text"}  # Added response_format field
        }

    async def create(self, messages: List[Dict[str, str]], **kwargs) -> Dict:
        """Create a chat completion using Gemini."""
        try:
            # Convert messages to Gemini format
            formatted_messages = []
            for msg in messages:
                # Handle function calls and results
                if msg.get("function_call"):
                    content = f"Function call: {json.dumps(msg['function_call'])}"
                elif msg.get("role") == "function":
                    content = f"Function result: {msg['content']}"
                else:
                    content = msg["content"]
                
                role = "user" if msg["role"] in ["user", "function"] else "model"
                formatted_messages.append({"role": role, "parts": [content]})
            
            # Get response from Gemini
            response = await self.chat.send_message_async(
                formatted_messages[-1]["parts"][0],
                generation_config=genai.types.GenerationConfig(
                    temperature=kwargs.get("temperature", self.config["temperature"]),
                    top_p=kwargs.get("top_p", self.config["top_p"]),
                    max_output_tokens=kwargs.get("max_tokens", self.model_info["max_output_tokens"])
                )
            )
            
            return {
                "choices": [{
                    "message": {
                        "content": response.text,
                        "role": "assistant"
                    }
                }]
            }
        except Exception as e:
            raise Exception(f"Error in Gemini API call: {str(e)}")

    def get_model_info(self):
        """Return model information."""
        return self.model_info

    def get_config(self):
        """Return configuration."""
        return self.config

# Initialize your tools (assuming google_search_tool and arxiv_search_tool are defined)
google_search_agent = AssistantAgent(
    name="Google_Search_Agent",
    tools=[google_search_tool],
    model_client=GeminiChatCompletionClient(model="gemini-pro"),
    description="An agent that can search Google for information, returns results with a snippet and body content",
    system_message="You are a helpful AI assistant. Solve tasks using your tools.",
)

arxiv_search_agent = AssistantAgent(
    name="Arxiv_Search_Agent",
    tools=[arxiv_search_tool],
    model_client=GeminiChatCompletionClient(model="gemini-pro"),
    description="An agent that can search Arxiv for papers related to a given topic, including abstracts",
    system_message="You are a helpful AI assistant. Solve tasks using your tools. Specifically, you can take into consideration the user's request and craft a search query that is most likely to return relevant academic papers.",
)

report_agent = AssistantAgent(
    name="Report_Agent",
    model_client=GeminiChatCompletionClient(model="gemini-pro"),
    description="Generate a report based on a given topic",
    system_message="You are a helpful assistant. Your task is to synthesize data extracted into a high quality literature review including CORRECT references. You MUST write a final report that is formatted as a literature review with CORRECT references. Your response should end with the word 'TERMINATE'",
)

# Your existing termination and team setup
termination = TextMentionTermination("TERMINATE")
team = RoundRobinGroupChat(
    participants=[google_search_agent, arxiv_search_agent, report_agent],
    termination_condition=termination
)

# For notebook environment, use this directly:
await Console(
    team.run_stream(
        task="Write a literature review on no code tools for building multi agent ai systems",
    )
)

Error processing publish message for Google_Search_Agent/0868cf6e-8324-4eba-aa9d-af8efbf0d3e4
Traceback (most recent call last):
  File "/var/folders/qz/jy_v70tj0792_gq7d947v1pw0000gn/T/ipykernel_26570/2364728106.py", line 51, in create
    if msg.get("function_call"):
       ^^^^^^^
  File "/Users/hissain/git/github/llm/llm_literature/.venv/lib/python3.11/site-packages/pydantic/main.py", line 891, in __getattr__
    raise AttributeError(f'{type(self).__name__!r} object has no attribute {item!r}')
AttributeError: 'SystemMessage' object has no attribute 'get'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/hissain/git/github/llm/llm_literature/.venv/lib/python3.11/site-packages/autogen_core/_single_threaded_agent_runtime.py", line 505, in _on_message
    return await agent.on_message(
           ^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/hissain/git/github/llm/llm_literature/.venv/lib/python3.11/site-packages/autogen_co

---------- user ----------
Write a literature review on no code tools for building multi agent ai systems


TaskResult(messages=[TextMessage(source='user', models_usage=None, content='Write a literature review on no code tools for building multi agent ai systems', type='TextMessage')], stop_reason=None)